<a href="https://colab.research.google.com/github/louis-not/Notogo-ML/blob/main/model_v2.2/Hyperparameter_Tuning_V2_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RUN

## Import Module

### For Optimization

In [ ]:
!pip install bayesian-optimization

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
from bayes_opt import BayesianOptimization

### For Model

In [ ]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

     |████████████████████████████████| 85 kB 2.2 MB/s 
     |████████████████████████████████| 462 kB 32.6 MB/s 
     |████████████████████████████████| 4.2 MB 4.5 MB/s 


In [ ]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

import pandas as pd

import tensorflow_recommenders as tfrs

# getting data
from google.colab import auth
import gspread
from google.auth import default
from gspread_dataframe import get_as_dataframe, set_with_dataframe

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/Bangkit/Capstone/ML/ML-GH/Notogo-ML/"model_v2.2"

/content/drive/MyDrive/Bangkit/Capstone/ML/ML-GH/Notogo-ML/model_v2.2


In [ ]:
print(os.getcwd())

/content/drive/MyDrive/Bangkit/Capstone/ML/ML-GH/Notogo-ML/model_v2.2


## Retrieval Model

### Preparing Dataset

In [ ]:
import userFeatures
builder = tfds.builder('Userfeatures')
userFeatureDs = tfds.load('Userfeatures',split='train')

import wishEmbedding
builder = tfds.builder('Wishembedding')
wishEmbeddingDs = tfds.load('Wishembedding',split='train')

ratings = userFeatureDs.map(lambda x: {
    "location_name": x["location_name"],
    "user_id": x["user_id"],
    "add" : x["add"],
    "like" : x['like']
})
locations = wishEmbeddingDs.map(lambda x: x["location_name"])

for x in ratings.take(2).as_numpy_iterator():
  pprint.pprint(x)

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/userfeatures/1.0.0.incomplete3699XW/userfeatures-train.tfrecord*...:   0%|…

Dataset userfeatures downloaded and prepared to /root/tensorflow_datasets/userfeatures/1.0.0. Subsequent calls will reuse this data.


Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/wishembedding/1.0.0.incompleteXUW5HS/wishembedding-train.tfrecord*...:   0…

Dataset wishembedding downloaded and prepared to /root/tensorflow_datasets/wishembedding/1.0.0. Subsequent calls will reuse this data.
{'add': 0, 'like': 1, 'location_name': b'JAKARTA', 'user_id': b'93'}
{'add': 0, 'like': 1, 'location_name': b'DUBAI', 'user_id': b'112'}


In [ ]:
tf.random.set_seed(42)

NUM_DATA = ratings.__len__().numpy()

shuffled = ratings.shuffle(NUM_DATA, seed=42, reshuffle_each_iteration=False)

trainset_size = 0.8 * NUM_DATA

train = shuffled.take(trainset_size)
test = shuffled.skip(trainset_size).take(NUM_DATA - trainset_size)

location_name = locations.batch(1000)
user_ids = ratings.batch(1000).map(lambda x: x["user_id"])

unique_location_name = np.unique(np.concatenate(list(location_name)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

cached_train = train.shuffle(NUM_DATA).batch(512).cache()
cached_test = test.batch(256).cache()

### Model
change this if needed for a different model

In [ ]:
class NoToGoModel(tfrs.models.Model):

  def __init__(self, rating_weight: float, like_weight: float,retrieval_weight: float, 
               BATCH_SIZE, EMB_DIM
               ) -> None:
    # We take the loss weights in the constructor: this allows us to instantiate
    # several model objects with different loss weights.

    super().__init__()

    embedding_dimension = int(EMB_DIM)

    # User and movie models.
    self.location_model: tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_location_name, mask_token=None),
      tf.keras.layers.Embedding(len(unique_location_name) + 1, embedding_dimension),
      tf.keras.layers.Dense(16, activation="relu")
    ])

    self.user_model: tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension),
      tf.keras.layers.Dense(16, activation="relu")
    ])

    # A small model to take in user and movie embeddings and predict ratings.
    # We can make this as complicated as we want as long as we output a scalar
    # as our prediction.
    self.rating_model = tf.keras.Sequential([
        tf.keras.layers.Dense(8, activation="relu"),
        tf.keras.layers.Dense(16, activation="relu"),
        tf.keras.layers.Dense(1, activation = "sigmoid"),
    ])

    self.like_model = tf.keras.Sequential([
        tf.keras.layers.Dense(16, activation="relu"),
        tf.keras.layers.Dense(32, activation="relu"),
        tf.keras.layers.Dense(1, activation = "sigmoid"),
    ])

    # The tasks.
    self.rating_task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
        loss=tf.keras.losses.MeanSquaredError(),
        metrics=[tf.keras.metrics.RootMeanSquaredError()],
    )

    self.like_task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
        loss=tf.keras.losses.MeanSquaredError(),
        metrics=[tf.keras.metrics.RootMeanSquaredError()],
    )
    self.retrieval_task: tf.keras.layers.Layer = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=locations.batch(BATCH_SIZE).map(self.location_model)
        )
    )

    # The loss weights.
    self.rating_weight = rating_weight
    self.retrieval_weight = retrieval_weight
    self.like_weight = like_weight

  def call(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["user_id"])
    # And pick out the movie features and pass them into the movie model.
    location_embeddings = self.location_model(features["location_name"])
    
    return (
        user_embeddings,
        location_embeddings,
        # We apply the multi-layered rating model to a concatentation of
        # user and movie embeddings.
        self.rating_model(
            tf.concat([user_embeddings,location_embeddings], axis=1)
        ),
        self.like_model(
            tf.concat([user_embeddings,location_embeddings], axis=1)
        ),
    )

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:

    ratings = features.pop("add","like")
    like = features.pop("like", "add")

    user_embeddings, location_embeddings, rating_predictions, like_predictions = self(features)

    # We compute the loss for each task.
    rating_loss = self.rating_task(
        labels=ratings,
        predictions=rating_predictions,
    )

    like_loss = self.like_task(
        labels=like,
        predictions=like_predictions,
    )
    retrieval_loss = self.retrieval_task(user_embeddings, location_embeddings)

    # And combine them using the loss weights.
    return (self.rating_weight * rating_loss
            + self.retrieval_weight * retrieval_loss + like_loss*self.like_weight)

## Bayes Optimization

In [ ]:
def black_box(lr,emb_dim, epoch):
  """black box function for optimization used for Retrieval-specialized model"""
  METRIC = 'factorized_top_k/top_1_categorical_accuracy'
  batch = 180


  # Initialize model
  model = NoToGoModel(rating_weight=0.0, like_weight = 0, retrieval_weight=1.0,
                      BATCH_SIZE=batch, EMB_DIM=emb_dim)
  model.compile(optimizer=tf.keras.optimizers.Adagrad(lr))

  # training model
  model.fit(cached_train, epochs=int(epoch), verbose=0)
  result = model.evaluate(cached_train, return_dict=True, verbose=0)

  return result[METRIC]

In [ ]:
param_bounds = {
    # 'batch' : (150, 200),
    'lr' : (0.001,0.1),
    'emb_dim' : (16, 128),
    'epoch' : (5, 25)
}

optimizer = BayesianOptimization(
    f=black_box,
    pbounds=param_bounds,
    random_state=1,
)

In [ ]:
optimizer.maximize(
    init_points=2, #How many steps of random exploration you want to perform. Random exploration can help by diversifying the exploration space.
    n_iter=50, #How many steps of bayesian optimization you want to perform. The more steps the more likely to find a good maximum you are.
)

|   iter    |  target   |  emb_dim  |   epoch   |    lr     |
-------------------------------------------------------------
|  1        |  0.00768  |  62.71    |  19.41    |  0.001011 |
|  2        |  0.01126  |  49.86    |  7.935    |  0.01014  |
|  3        |  0.2441   |  50.12    |  7.721    |  0.05658  |
|  4        |  0.0104   |  75.62    |  20.75    |  0.01832  |
|  5        |  0.7548   |  82.73    |  10.22    |  0.07493  |
|  6        |  0.7947   |  41.48    |  8.112    |  0.09478  |
|  7        |  0.009021 |  41.84    |  7.695    |  0.05743  |
|  8        |  0.2844   |  41.37    |  8.098    |  0.058    |
|  9        |  0.2262   |  123.0    |  18.28    |  0.0476   |
|  10       |  0.7433   |  70.74    |  21.82    |  0.09793  |
|  11       |  0.1297   |  52.12    |  20.27    |  0.08584  |
|  12       |  0.3073   |  111.8    |  19.39    |  0.09129  |
|  13       |  0.02056  |  91.84    |  15.96    |  0.03894  |
|  14       |  0.0117   |  31.76    |  7.788    |  0.03237  |
|  15   

In [ ]:
print(optimizer.max)

{'target': 0.7947092652320862, 'params': {'emb_dim': 41.481665216522984, 'epoch': 8.111799947764348, 'lr': 0.09478304165663956}}


## Evaluate

In [ ]:
# Result :
batch = 180
emb_dim = 41
epoch = 8
lr = 0.09478

In [ ]:
model = NoToGoModel(rating_weight=0.0, like_weight = 0, retrieval_weight=1.0,
                    BATCH_SIZE=batch, EMB_DIM=emb_dim)
model.compile(optimizer=tf.keras.optimizers.Adagrad(lr))

# training model
model.fit(cached_train, epochs=int(epoch))
model.evaluate(cached_train, return_dict=True)

Epoch 1/8
49/49 [==============================] - 7s 79ms/step - root_mean_squared_error: 0.5012 - factorized_top_k/top_1_categorical_accuracy: 0.2154 - factorized_top_k/top_5_categorical_accuracy: 0.6207 - factorized_top_k/top_10_categorical_accuracy: 0.7907 - factorized_top_k/top_50_categorical_accuracy: 0.9234 - factorized_top_k/top_100_categorical_accuracy: 0.9802 - loss: 3070.9090 - regularization_loss: 0.0000e+00 - total_loss: 3070.9090
Epoch 2/8
49/49 [==============================] - 4s 80ms/step - root_mean_squared_error: 0.5011 - factorized_top_k/top_1_categorical_accuracy: 0.2903 - factorized_top_k/top_5_categorical_accuracy: 0.6783 - factorized_top_k/top_10_categorical_accuracy: 0.8558 - factorized_top_k/top_50_categorical_accuracy: 0.9991 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 3070.8701 - regularization_loss: 0.0000e+00 - total_loss: 3070.8701
Epoch 3/8
49/49 [==============================] - 4s 76ms/step - root_mean_squared_error: 0.5003 - fact

{'factorized_top_k/top_100_categorical_accuracy': 1.0,
 'factorized_top_k/top_10_categorical_accuracy': 0.9950830936431885,
 'factorized_top_k/top_1_categorical_accuracy': 0.4392701983451843,
 'factorized_top_k/top_50_categorical_accuracy': 1.0,
 'factorized_top_k/top_5_categorical_accuracy': 0.994920551776886,
 'loss': 115.38475036621094,
 'regularization_loss': 0,
 'root_mean_squared_error': 0.5000376105308533,
 'total_loss': 115.38475036621094}

In [ ]:
model.evaluate(cached_train, return_dict=True)

49/49 [==============================] - 4s 76ms/step - root_mean_squared_error: 0.5000 - factorized_top_k/top_1_categorical_accuracy: 0.4393 - factorized_top_k/top_5_categorical_accuracy: 0.9949 - factorized_top_k/top_10_categorical_accuracy: 0.9951 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 3070.5869 - regularization_loss: 0.0000e+00 - total_loss: 3070.5869


{'factorized_top_k/top_100_categorical_accuracy': 1.0,
 'factorized_top_k/top_10_categorical_accuracy': 0.9950830936431885,
 'factorized_top_k/top_1_categorical_accuracy': 0.4392701983451843,
 'factorized_top_k/top_50_categorical_accuracy': 1.0,
 'factorized_top_k/top_5_categorical_accuracy': 0.994920551776886,
 'loss': 115.38475036621094,
 'regularization_loss': 0,
 'root_mean_squared_error': 0.5000376105308533,
 'total_loss': 115.38475036621094}

In [ ]:
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
# recommends movies out of the entire movies dataset.
index.index_from_dataset(
  tf.data.Dataset.zip((locations.batch(100), locations.batch(100).map(model.location_model)))
)

# Get recommendations.
user_id = "183"
_, titles = index(tf.constant([user_id]))
print(f"Recommendations for New User : {titles[0, :7]}")

Recommendations for New User : [b'MUSEUM MACAN' b'PULAU SERIBU' b'PRANCIS' b'MEDAN' b'TOKYO'
 b'BALI TOWER' b'GEOPARK CILETUH']
